# Project 2 “Why did I get the flu?”. Deep sequencing, error control, p-value, viral evolution..


#### 15.11.24  Rakhimov Bulat

## Prepare workplace

In [36]:
# ~/.../Project2
!mkdir data

mkdir: невозможно создать каталог «data»: Файл существует


## Prepare the raw sequencing data

Sequencing results are usually stored in the NCBI Sequence Read Archive, or SRA. Your roommate’s data was published there and labeled SRR1705851, so you can download it from the SRA FTP server and unpack it on your machine: http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/001/SRR1705851/



In [70]:
!wget https://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/001/SRR1705851/SRR1705851.fastq.gz 
!mv ./SRR1705851.fastq.gz ./data/SRR1705851.fastq.gz

--2024-11-22 23:03:33--  https://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/001/SRR1705851/SRR1705851.fastq.gz
Распознаётся ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)… 193.62.193.165
Подключение к ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.165|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 19217176 (18M) [application/x-gzip]
Сохранение в: «SRR1705851.fastq.gz.1»

SRR1705851.fastq.gz 100%[===================>]  18,33M   903KB/s    за 48s     

2024-11-22 23:04:30 (391 KB/s) - «SRR1705851.fastq.gz.1» сохранён [19217176/19217176]



Open the sequence files and verify that the format is correct.

In [71]:
!zless ./data/SRR1705851.fastq.gz | head -20

@SRR1705851.1 1/1
TTCGTGATTGTTTTCACTATCGTTCCGTTTGGCACTGCATGGTGCCCAAGGCACAGCGTTGCCGTGCTGTTGTCATTTCCAGGAAGTTTTTGAGCGAAAACCAGACATAGAATGTAGCTCAAAGCAATGATAGTCTTCATGGTTAATAG
+
,<==<<<<A@@@@@@@EEE;CEE+AC>EC;>EFFDC@=A@AE999DDD>>@E777EE75C>EF>EDEEFFFF--AE>EDEEEED=C-58AE=<D=<<DD=D9CDD@EEDED@DEDDE*9;@DDED@@@7@E*;*888@*8;@8@;;@@E
@SRR1705851.2 2/1
NATTAACCATGAAGACTATCATTGCTTTGAGCTACATTCTATGTCTGGTTTTCGCTCAAAAACTTCCTGGAAATGACAACAGCACGGCAACGCTGTGCCTTGGGCACCATGCAGTGCCAAACGGAACGATAGTGAAAACAATCACGAATGA
+
#5<???BBEEEDEDDDGGGGGGIIIIIIIIIIIIIIIIIIIIIHIIIIFHHIIHHHHHIIIIHIIIIIIIHIIIIIIIIIIIIIIHHHHHHHHHHEHHHHHFFHHHHHHFFHHGFGGGGGGGGGGGGGEEEGCEEGGGGGEEGGGGCGEGG
@SRR1705851.3 3/1
GTTTGGCACTGCATGGTGCCCAAGGCACAGCGTTGCCGTGCTGTTGTCATTTCCAGGAAGTTTTTGAGCGAAAACCAGACATAGAATGTAGCTCAAAGCAATGATAGTCTTCATGGTTAATAG
+
9=9===>>@@@--5-588C;C;6+AE>6-A@+7>++-+@C+A.ECFE..8E.8--AE+C,EA@FEC5+-**55C-++++++++==D+5+,,+++++6CEDE====C=3C:@D+2++2<:DE2*
@SRR1705851.4 4/1
GTGCCCAAGGCACAGCGTTGCCGTGCTGTTGTCATTTCCAGGAAGTTTTTGAGCGAAAACCAGACATAGA

To see how many lines there are in each fastq file

In [72]:
!zless ./data/SRR1705851.fastq.gz | grep -c '@SRR' 

358265


In [73]:
!zcat ./data/SRR1705851.fastq.gz | wc -l

1433060


In [66]:
1433060 / 4

358265.0

## Read processing 

### Quality control

In [5]:
!mkdir ./data/fastqc_report

mkdir: невозможно создать каталог «./data/fastqc_report»: Файл существует


In [74]:
!fastqc -o ./data/fastqc_report ./data/SRR1705851.fastq.gz --threads 12

application/gzip
Started analysis of SRR1705851.fastq.gz
Approx 5% complete for SRR1705851.fastq.gz
Approx 10% complete for SRR1705851.fastq.gz
Approx 15% complete for SRR1705851.fastq.gz
Approx 20% complete for SRR1705851.fastq.gz
Approx 25% complete for SRR1705851.fastq.gz
Approx 30% complete for SRR1705851.fastq.gz
Approx 35% complete for SRR1705851.fastq.gz
Approx 40% complete for SRR1705851.fastq.gz
Approx 45% complete for SRR1705851.fastq.gz
Approx 50% complete for SRR1705851.fastq.gz
Approx 55% complete for SRR1705851.fastq.gz
Approx 60% complete for SRR1705851.fastq.gz
Approx 65% complete for SRR1705851.fastq.gz
Approx 70% complete for SRR1705851.fastq.gz
Approx 75% complete for SRR1705851.fastq.gz
Approx 80% complete for SRR1705851.fastq.gz
Approx 85% complete for SRR1705851.fastq.gz
Approx 90% complete for SRR1705851.fastq.gz
Approx 95% complete for SRR1705851.fastq.gz
Analysis complete for SRR1705851.fastq.gz


### Reads filtering

In [75]:
!trimmomatic SE -phred33 ./data/SRR1705851.fastq.gz ./data/SRR1705851_trim.fastq LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:30

TrimmomaticSE: Started with arguments:
 -phred33 ./data/SRR1705851.fastq.gz ./data/SRR1705851_trim.fastq LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:30
Automatically using 1 threads
Input Reads: 358265 Surviving: 356827 (99,60%) Dropped: 1438 (0,40%)
TrimmomaticSE: Completed successfully


### Repeat quality control

In [76]:
!fastqc -o ./data/fastqc_report ./data/SRR1705851_trim.fastq --threads 12

null
Started analysis of SRR1705851_trim.fastq
Approx 5% complete for SRR1705851_trim.fastq
Approx 10% complete for SRR1705851_trim.fastq
Approx 15% complete for SRR1705851_trim.fastq
Approx 20% complete for SRR1705851_trim.fastq
Approx 25% complete for SRR1705851_trim.fastq
Approx 30% complete for SRR1705851_trim.fastq
Approx 35% complete for SRR1705851_trim.fastq
Approx 40% complete for SRR1705851_trim.fastq
Approx 45% complete for SRR1705851_trim.fastq
Approx 50% complete for SRR1705851_trim.fastq
Approx 55% complete for SRR1705851_trim.fastq
Approx 60% complete for SRR1705851_trim.fastq
Approx 65% complete for SRR1705851_trim.fastq
Approx 70% complete for SRR1705851_trim.fastq
Approx 75% complete for SRR1705851_trim.fastq
Approx 80% complete for SRR1705851_trim.fastq
Approx 85% complete for SRR1705851_trim.fastq
Approx 90% complete for SRR1705851_trim.fastq
Approx 95% complete for SRR1705851_trim.fastq
Analysis complete for SRR1705851_trim.fastq


## Prepare reference HA gene sequence¶

Download reference HA gene sequence using `EntrezDirect` utility

In [1]:
''' ADVANCED '''
#!conda install bioconda::entrez-direct
!efetch -db nucleotide -id KF848938.1 -format fasta > ./data/HA_ref.fasta

In [10]:
!mkdir ./aln 
!mkdir ./ref

mkdir: невозможно создать каталог «./aln»: Файл существует
mkdir: невозможно создать каталог «./ref»: Файл существует


In [2]:
!bwa index ./data/HA_ref.fasta -p ./ref/HA_ref

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.18-r1243-dirty
[main] CMD: bwa index -p ./ref/HA_ref ./data/HA_ref.fasta
[main] Real time: 0.017 sec; CPU: 0.016 sec


In [3]:
''' ADVANCED '''
!bwa mem ./ref/HA_ref ./data/SRR1705851_trim.fastq | samtools view -S -b - | samtools sort -o ./aln/alignment.bam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 69192 sequences (10000108 bp)...
[M::process] read 68586 sequences (10000065 bp)...
[M::mem_process_seqs] Processed 69192 reads in 2.495 CPU sec, 2.401 real sec
[M::process] read 68414 sequences (10000042 bp)...
[M::mem_process_seqs] Processed 68586 reads in 1.908 CPU sec, 1.805 real sec
[M::process] read 68546 sequences (10000082 bp)...
[M::mem_process_seqs] Processed 68414 reads in 2.451 CPU sec, 2.351 real sec
[M::process] read 69188 sequences (10000076 bp)...
[M::mem_process_seqs] Processed 68546 reads in 2.298 CPU sec, 2.202 real sec
[M::process] read 12901 sequences (1795937 bp)...
[M::mem_process_seqs] Processed 69188 reads in 1.233 CPU sec, 1.190 real sec
[M::mem_process_seqs] Processed 12901 reads in 0.318 CPU sec, 0.291 real sec
[main] Version: 0.7.18-r1243-dirty
[main] CMD: bwa mem ./ref/HA_ref ./data/SRR1705851_trim.fastq
[main] Real time: 10.434 sec; CPU: 10.815 sec


In [80]:
!samtools index ./aln/alignment.bam

In [7]:
!samtools view -c -F 2308 ./aln/alignment.bam

356434


In [81]:
!samtools flagstat ./aln/alignment.bam

359849 + 0 in total (QC-passed reads + QC-failed reads)
356827 + 0 primary
0 + 0 secondary
3022 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
359456 + 0 mapped (99.89% : N/A)
356434 + 0 primary mapped (99.89% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [51]:
# It’s good to check that we selected correct reference - we can count the unmapped reads that mapped
!bwa mem ./ref/HA_ref ./data/SRR1705851_trim.fastq > ./aln/alignment.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 69192 sequences (10000108 bp)...
[M::process] read 68586 sequences (10000065 bp)...
[M::mem_process_seqs] Processed 69192 reads in 2.408 CPU sec, 2.331 real sec
[M::process] read 68414 sequences (10000042 bp)...
[M::mem_process_seqs] Processed 68586 reads in 1.903 CPU sec, 1.795 real sec
[M::process] read 68546 sequences (10000082 bp)...
[M::mem_process_seqs] Processed 68414 reads in 2.458 CPU sec, 2.354 real sec
[M::process] read 69188 sequences (10000076 bp)...
[M::mem_process_seqs] Processed 68546 reads in 2.376 CPU sec, 2.280 real sec
[M::process] read 12901 sequences (1795937 bp)...
[M::mem_process_seqs] Processed 69188 reads in 1.586 CPU sec, 1.512 real sec
[M::mem_process_seqs] Processed 12901 reads in 0.310 CPU sec, 0.274 real sec
[main] Version: 0.7.18-r1243-dirty
[main] CMD: bwa mem ./ref/HA_ref ./data/SRR1705851_trim.fastq
[main] Real time: 10.684 sec; CPU: 11.156 sec


In [52]:
!samtools view -c -f 4 ./aln/alignment.sam

393


In [17]:
356827 - 393

356434

Сalculate reference coverage by reads:

coverage = (read count * read length ) / total genome size

In [18]:
# average read length
(30 + 151) / 2

90.5

In [19]:
# coverage
(356434 * 90.5) / 1665

19373.73993993994

## Look for common variants

The goal now is to go through our data, and for each position in the reference, see how many reads have a mutation at the same position - we want to distinguish actual mutations from the sequencing errors. The solution is to make an intermediate file type called an mpileup, because it goes through each position and “piles up” the reads, tabulating the number of bases that match or don’t match the reference.

To save computing power, the default behavior of samtools is to stop piling up the base calls at each position when it gets to 8000 calls. Since our variants may be quite rare, set that depth limit to something we know is higher than our coverage with the -d flag.

In [20]:
!mkdir ./variants

mkdir: невозможно создать каталог «./variants»: Файл существует


In [53]:
!samtools mpileup -d 20000 -f ./data/HA_ref.fasta ./aln/alignment.bam > ./variants/flu.mpileup	

[mpileup] 1 samples in 1 input files


First, look for positions where most of the viruses infecting your roommate differ from the reference. Maybe there is a common mutation that wouldn’t have shown up in the HI test. We use a high minimum variant frequency cut-off (N) to find only those mutants present in most (95% or more = 0.95) of the viral DNA molecules.


In [54]:
!varscan mpileup2snp ./variants/flu.mpileup --min-var-freq 0.95 --variants --output-vcf 1 > ./variants/flu_VarScan_0.95.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.95
Min avg qual:	15
P-value thresh:	0.01
Reading input from ./variants/flu.mpileup
1665 bases in pileup file
5 variant positions (5 SNP, 0 indel)
0 were failed by the strand-filter
5 variant positions reported (5 SNP, 0 indel)


We can pull out the variants in a convenient format using awk or any parsing tool of your choice. Awk excels at parsing delimited data where you have a lot of fields, like in a vcf file.
The simplest form of an awk command is

We just need to extract some particular columns from our file. awk script below only output the fields you are interested in (reference, position, alternate). 




In [55]:
cat ./variants/flu_VarScan_0.95.vcf | awk 'NR>24 {print $1, $2, $4, $5}'

KF848938.1 72 A G
KF848938.1 117 C T
KF848938.1 774 T C
KF848938.1 999 C T
KF848938.1 1260 A C


Now try looking for rare variants. Set the minimum variant frequency to 0.001 (0.1%) and run the scan again on the same mpileup file. How many variants are reported back now, and how abundant are they? There is a value in the FREQ field buried deeply in the 10th column of vcf file - you can eyeball it or, again, pull it out with awk.

In [56]:
!varscan mpileup2snp ./variants/flu.mpileup --min-var-freq 0.001 --variants --output-vcf 1 > ./variants/flu_VarScan_0.001.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.001
Min avg qual:	15
P-value thresh:	0.01
Reading input from ./variants/flu.mpileup
1665 bases in pileup file
20 variant positions (18 SNP, 2 indel)
0 were failed by the strand-filter
18 variant positions reported (18 SNP, 0 indel)


In [57]:
cat ./variants/flu_VarScan_0.001.vcf | awk 'NR>24 { split($10, subfield, ":"); print $1, $2, $4, $5, subfield[7] }'


KF848938.1 72 A G 99.94%
KF848938.1 117 C T 99.73%
KF848938.1 165 T C 0.2%
KF848938.1 254 A G 0.2%
KF848938.1 276 A G 0.19%
KF848938.1 307 C T 0.9%
KF848938.1 389 T C 0.23%
KF848938.1 722 A G 0.22%
KF848938.1 774 T C 99.96%
KF848938.1 802 A G 0.21%
KF848938.1 913 T C 0.2%
KF848938.1 915 T C 0.24%
KF848938.1 999 C T 99.86%
KF848938.1 1086 A G 0.24%
KF848938.1 1213 A G 0.26%
KF848938.1 1260 A C 99.92%
KF848938.1 1280 T C 0.19%
KF848938.1 1458 T C 0.81%


In [58]:
# save to file
!cat ./variants/flu_VarScan_0.001.vcf | awk 'NR>24 { split($10, subfield, ":"); print $1, $2, $4, $5, subfield[7] }'  > ./variants/flu_muts.txt

It’s very difficult to tell the difference between real, rare mutants in the viral population, and errors introduced in the sequencing and amplification process.

We can take a control reaction with an isogenic viral sample (all virus particles genetically identical) derived from a virus clone that matches the reference sequence (H3N2 influenza virus). By looking at the errors in this reference and comparing them to the mutations in your roommate’s sample, you hope you’ll be able to figure out which variants are real.

Any “mutations” you detect in the control samples which don’t contain any true genetic variants must be due to errors. You can use the frequency of the errors from the control to help figure out what’s an error and what’s a true variant in the data from your roommate.

## Prepare the control sequencing data

In [27]:
!mkdir ./control_data

mkdir: невозможно создать каталог «./control_data»: Файл существует


In [28]:
#!conda install bioconda::parallel-fastq-dump
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/008/SRR1705858/SRR1705858.fastq.gz 
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/009/SRR1705859/SRR1705859.fastq.gz 
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/000/SRR1705860/SRR1705860.fastq.gz 

--2024-11-22 16:36:42--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/008/SRR1705858/SRR1705858.fastq.gz
           => «SRR1705858.fastq.gz»
Распознаётся ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)… 193.62.193.165
Подключение к ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.165|:21... соединение установлено.
Выполняется вход под именем anonymous … Выполнен вход в систему!
==> SYST ... готово.  ==> PWD ... готово.
==> TYPE I ... готово.   ==> CWD (1) /vol1/fastq/SRR170/008/SRR1705858 ... готово.
==> SIZE SRR1705858.fastq.gz ... 14782243
==> PASV ... готово.  ==> RETR SRR1705858.fastq.gz ... готово.
Размер (байт): 14782243 (14M) (не достоверно)

SRR1705858.fastq.gz 100%[===================>]  14,10M  1,77MB/s    за 8,3s    

2024-11-22 16:36:53 (1,69 MB/s) - «SRR1705858.fastq.gz» сохранён [14782243]

--2024-11-22 16:36:53--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/009/SRR1705859/SRR1705859.fastq.gz
           => «SRR1705859.fastq.gz»
Распознаётся ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)… 193.62.

In [29]:
!mv SRR* ./control_data

In [ ]:
from glob import glob
import os

wrkdir = '/home/bulat/IB-2024-2025/Practbioinf/Project2'

os.makedirs('./control_aln',  exist_ok=True) # make control alignment directory
os.makedirs('./control_variants',  exist_ok=True)
os.chdir(wrkdir+'/control_data') # go to directory with control sequncing data
os.makedirs('./fastqc_report',  exist_ok=True)

for fastq_file in enumerate(sorted(glob('*.fastq.gz'))):
    
    prefix = fastq_file[1].split('.')[0]
    
    cmd1 = f'fastqc -o ./fastqc_report {fastq_file[1]} --threads 12'
    os.system(cmd1)
    
    cmd2 = f'trimmomatic SE -phred33 {fastq_file[1]} {prefix}_trim.fastq LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:30'
    os.system(cmd2)
    
    cmd3 = f'fastqc -o ./fastqc_report {prefix}_trim.fastq --threads 12'
    os.system(cmd3)
    
    cmd4 = f'bwa mem ../ref/HA_ref {prefix}_trim.fastq | samtools view -S -b - | samtools sort -o ../control_aln/{prefix}_alignment.bam'
    os.system(cmd4)
    
    cmd5 = f'samtools flagstat ../control_aln/{prefix}_alignment.bam'
    os.system(cmd5)
    
    cmd6 = f'samtools sort ../control_aln/{prefix}_alignment.bam -o ../control_aln/{prefix}_alignment_sorted.bam'
    os.system(cmd6)
    
    cmd7 = f'samtools index ../control_aln/{prefix}_alignment_sorted.bam'
    os.system(cmd7)
    
    cmd8 = f'samtools mpileup -d 20000 -f ../ref/HA_ref.fasta ../control_aln/{prefix}_alignment_sorted.bam > ../control_variants/{prefix}.mpileup'
    os.system(cmd8)
    
    cmd9 = f'varscan mpileup2snp ../control_variants/{prefix}.mpileup --min-var-freq 0.001 --variants --output-vcf 1 > ../control_variants/{prefix}_VarScan_0.001.vcf'
    os.system(cmd9)

    
    cmd10 = f"cat ../control_variants/{prefix}_VarScan_0.001.vcf" + " | awk 'NR>24 { split($10, subfield, \":\"); print $1, $2, $4, $5, subfield[7] }'" + f" > ../control_variants/{prefix}_muts.txt"
    os.system(cmd10)

os.chdir(wrkdir)

application/gzip


Started analysis of SRR1705851.fastq.gz
Approx 5% complete for SRR1705851.fastq.gz
Approx 10% complete for SRR1705851.fastq.gz
Approx 15% complete for SRR1705851.fastq.gz
Approx 20% complete for SRR1705851.fastq.gz
Approx 25% complete for SRR1705851.fastq.gz
Approx 30% complete for SRR1705851.fastq.gz
Approx 35% complete for SRR1705851.fastq.gz
Approx 40% complete for SRR1705851.fastq.gz
Approx 45% complete for SRR1705851.fastq.gz
Approx 50% complete for SRR1705851.fastq.gz
Approx 55% complete for SRR1705851.fastq.gz
Approx 60% complete for SRR1705851.fastq.gz
Approx 65% complete for SRR1705851.fastq.gz
Approx 70% complete for SRR1705851.fastq.gz
Approx 75% complete for SRR1705851.fastq.gz
Approx 80% complete for SRR1705851.fastq.gz
Approx 85% complete for SRR1705851.fastq.gz
Approx 90% complete for SRR1705851.fastq.gz
Approx 95% complete for SRR1705851.fastq.gz


Analysis complete for SRR1705851.fastq.gz


TrimmomaticSE: Started with arguments:
 -phred33 SRR1705851.fastq.gz SRR1705851_trim.fastq LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:30
Automatically using 1 threads
Input Reads: 358265 Surviving: 356827 (99,60%) Dropped: 1438 (0,40%)
TrimmomaticSE: Completed successfully


null


Started analysis of SRR1705851_trim.fastq
Approx 5% complete for SRR1705851_trim.fastq
Approx 10% complete for SRR1705851_trim.fastq
Approx 15% complete for SRR1705851_trim.fastq
Approx 20% complete for SRR1705851_trim.fastq
Approx 25% complete for SRR1705851_trim.fastq
Approx 30% complete for SRR1705851_trim.fastq
Approx 35% complete for SRR1705851_trim.fastq
Approx 40% complete for SRR1705851_trim.fastq
Approx 45% complete for SRR1705851_trim.fastq
Approx 50% complete for SRR1705851_trim.fastq
Approx 55% complete for SRR1705851_trim.fastq
Approx 60% complete for SRR1705851_trim.fastq
Approx 65% complete for SRR1705851_trim.fastq
Approx 70% complete for SRR1705851_trim.fastq
Approx 75% complete for SRR1705851_trim.fastq
Approx 80% complete for SRR1705851_trim.fastq
Approx 85% complete for SRR1705851_trim.fastq
Approx 90% complete for SRR1705851_trim.fastq
Approx 95% complete for SRR1705851_trim.fastq


Analysis complete for SRR1705851_trim.fastq


In [62]:
import pandas as pd 
from glob import glob
import os 

wrkdir = '/home/bulat/IB-2024-2025/Practbioinf/Project2'
os.chdir(wrkdir + '/control_variants')

for muts_file in enumerate(sorted(glob('*_muts.txt'))):
    data = pd.read_csv(muts_file[1], header=None, index_col=None, sep=' ')
    
    # Убираем символы процента и запятые, затем преобразуем в float
    data[4] = data[4].replace({'%': ''}, regex=True).astype(float)
    
    # Печатаем имя файла (prefix) и статистику для 4-го столбца
    print(muts_file[1].split('_')[0])
    print(data[4].describe())  # Статистика для 4-го столбца
    print()

os.chdir(wrkdir)


SRR1705851
count    18.000000
mean     27.972222
std      45.889943
min       0.190000
25%       0.202500
50%       0.240000
75%      75.022500
max      99.960000
Name: 4, dtype: float64

SRR1705858
count    52.000000
mean      0.262308
std       0.071335
min       0.190000
25%       0.220000
50%       0.250000
75%       0.282500
max       0.660000
Name: 4, dtype: float64

SRR1705859
count    43.000000
mean      0.247209
std       0.053778
min       0.190000
25%       0.210000
50%       0.240000
75%       0.250000
max       0.480000
Name: 4, dtype: float64

SRR1705860
count    52.000000
mean      0.263462
std       0.081286
min       0.190000
25%       0.220000
50%       0.250000
75%       0.272500
max       0.700000
Name: 4, dtype: float64



In [32]:
0.280000 + (0.079561 * 3)

0.518683

In [33]:
0.263939 + (0.055730 * 3)

0.431129

In [34]:
0.280000 + (0.086842 * 3)

0.5405260000000001

In [63]:
import pandas as pd 

THRESHOLD = 0.54

data = pd.read_csv('./variants/flu_muts.txt', header=None, index_col=None, sep=' ')
data[4] = data[4].replace({'%': ''}, regex=True).astype(float)
data.drop(data[data[4] <= THRESHOLD].index)

,0,1,2,3,4
0,KF848938.1,72,A,G,99.94
1,KF848938.1,117,C,T,99.73
5,KF848938.1,307,C,T,0.90
8,KF848938.1,774,T,C,99.96
12,KF848938.1,999,C,T,99.86
15,KF848938.1,1260,A,C,99.92
17,KF848938.1,1458,T,C,0.81


307 мутация C -> T находится в эпитопе С. Пролин меняется на серин (P -> S)